(sec-remote-memory-access)=
# 远程内存访问

{numref}`sec-mpi-hello-world` 中我们介绍了两种通信模式，即双边和单边。前几章节的点对点通信和集合通信主要针对的双边通信，本节主要介绍单边通信。单边通信是进程间直接访问远程内存，又称为远程内存访问（Remote Memory Access，RMA）。

## Window

任何进程所分配的内存都是私有的，即只被进程自己访问，远程内存访问要把进程自己的内存区域暴露给其他进程访问，这部分内存不再私有，而变成了公共的，需要特别处理。在 MPI 中，使用窗口（Window）定义可被远程访问的内存区域。某个内存区域被设置为允许远程访问，所有 Window 内的进程都可对这个区域进行读写。{numref}`fig-rma-window` 展示了私有的内存区域和可被远程访问的 Window。mpi4py 提供了 [`mpi4py.MPI.Win`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Win.html#mpi4py.MPI.Win) 类进行 Window 相关操作。

```{figure} ../img/ch-mpi/rma-window.svg
---
width: 600px
name: fig-rma-window
---
进程私有内存与允许远程访问的 Window
```

## 创建 Window

我们可以使用 [`mpi4py.MPI.Win.Allocate`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Win.html#mpi4py.MPI.Win.Allocate) 和 [`mpi4py.MPI.Win.Create`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Win.html#mpi4py.MPI.Win.Create) 创建 Window。其中 `mpi4py.MPI.Win.Allocate` 创建新的内存缓存，并且该缓存可被远程访问；`mpi4py.MPI.Win.Create` 将现有的某个内存缓存区域设置为可远程访问。这个区别主要体现在这两个方法的第一个参数，`mpi4py.MPI.Win.Allocate(size)` 传入的要创建内存缓存的字节数 `size`； `mpi4py.MPI.Win.Create(memory)` 传入的是内存地址 `memory`。

## 读写操作

创建好可远程访问的 Window 后，可以使用三类方法来向内存区域读写数据：[`mpi4py.MPI.Win.Put`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Win.html#mpi4py.MPI.Win.Put)，[`mpi4py.MPI.Win.Get`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Win.html#mpi4py.MPI.Win.Get) 和 [`mpi4py.MPI.Win.Accumulate`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Win.html#mpi4py.MPI.Win.Accumulate)。这三类方法都有两个参数：`origin` 和 `target_rank`，分别表示源进程和目标进程。源进程指的是调用读写方法的进程，目标进程是被读写的进程。

* `Win.Put` 将数据从源进程移动至目标进程。
* `Win.Get` 将数据从目标进程移动至源进程。
* `Win.Accumulate` 与 `Win.Put` 类似，也是将数据从源进程移动至目标进程，同时对源进程的数据和目标进程的数据进行了聚合操作，聚合操作的操作符包括：[`mpi4py.MPI.SUM`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.SUM.html)、[`mpi4py.MPI.PROD`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.PROD.html) 等。

## 数据同步

单机程序是顺序执行的，多机环境下，因为涉及多方数据的读写，可能会出现一些数据同步的问题，如 {numref}`fig-rma-sync-problem` 所示，如果不明确读写操作的顺序，尤其是短时间内同时执行 PUT 会导致某块内存区域的数据并非为程序员所期望的结果。

```{figure} ../img/ch-mpi/rma-sync-problem.svg
---
width: 600px
name: fig-rma-sync-problem
---
并行数据读写时会出现数据同步的问题，P0 和 P1 表示两个不同的进程。
```

为解决这个问题，需要一定的数据同步机制。MPI 一共有几类，包括主动同步（Active Target Synchronization）和被动同步（Passive Target Synchronization），如 {numref}`fig-rma-synchronization` 所示。

```{figure} ../img/ch-mpi/rma-synchronization.svg
---
width: 600px
name: fig-rma-synchronization
---
主动同步与被动同步
```

## 案例：远程读写

一个完整的 RMA 程序应该包括：

1. 创建 Window
2. 数据同步
3. 数据读写

{numref}`code-mpi-rma-lock` 展示了一个案例，其代码保存为 `rma-lock.py`。

```{code-block} python
:caption: rma-lock.py
:name: code-mpi-rma-lock

import numpy as np
from mpi4py import MPI
from mpi4py.util import dtlib

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

datatype = MPI.FLOAT
np_dtype = dtlib.to_numpy_dtype(datatype)
itemsize = datatype.Get_size()

N = 8
win_size = N * itemsize if rank == 0 else 0
win = MPI.Win.Allocate(win_size, comm=comm)

buf = np.empty(N, dtype=np_dtype)
if rank == 0:
    buf.fill(42)
    win.Lock(rank=0)
    win.Put(buf, target_rank=0)
    win.Unlock(rank=0)
    comm.Barrier()
else:
    comm.Barrier()
    win.Lock(rank=0)
    win.Get(buf, target_rank=0)
    win.Unlock(rank=0)
    if np.all(buf == 42):
        print(f"win.Get successfully on Rank {comm.Get_rank()}.")
    else:
        print(f"win.Get failed on Rank {comm.Get_rank()}.")
```

In [3]:
!mpiexec -np 8 python rma_lock.py

win.Get successfully on Rank 4.
win.Get successfully on Rank 5.
win.Get successfully on Rank 6.
win.Get successfully on Rank 7.
win.Get successfully on Rank 1.
win.Get successfully on Rank 2.
win.Get successfully on Rank 3.
